In [1]:
import os
os.chdir(os.path.join(os.getcwd(), '..'))

In [2]:
import torch
from tqdm import tqdm
import yaml
from models.bendr import EncoderConv, ContextNetwork

with open('configs/od_config_bendr.yaml', 'r') as file:
        cfg = yaml.safe_load(file)


encoder_sd = torch.load('/Users/ksc/Desktop/Нейротемка/checkpoints_70ep/encoder_2024-12-23T19:57:53.101820.pt', map_location=torch.device('cpu') )
context_network_sd = torch.load('/Users/ksc/Desktop/Нейротемка/checkpoints_70ep/context_network_2024-12-23T19:57:53.101820.pt', map_location=torch.device('cpu') )


encoder = EncoderConv(**cfg['encoder'])
context_network = ContextNetwork(**cfg['context_network'])

encoder.load_state_dict(encoder_sd)
context_network.load_state_dict(context_network_sd)
encoder.eval()
context_network.eval()

/var/folders/z2/jk8j2nn905v4_76cbzdgkrxm0000gn/T/ipykernel_24147/711149925.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder_sd = torch.load('/Users/ksc/Desktop/Н

ContextNetwork(
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-19): 20 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (target_proj): Linear(in_features=512, out_features=512, bias=True)
  (positional_emb): SinusoidalPositionalEncoding(
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=False)
  )
)

In [3]:
from dataset.labeled_dataset import EEGLabeledDataset

dataset = EEGLabeledDataset(
    data_path='/Users/ksc/Desktop/Нейротемка/music_data/h5',
    cache_processed_path='/Users/ksc/Desktop/Нейротемка/music_data/cache',
    train_length=73728,
    dataset_mode='sequential',
    target_config={
        'user_id': True,
        'activity': True
    },
    clipped_threshold=100
)

Building cache...


One chunk from each file:  29%|██▊       | 2/7 [00:00<00:01,  4.27it/s]

Skipping last 25752
Skipping last 67992


One chunk from each file:  43%|████▎     | 3/7 [00:00<00:01,  3.83it/s]

Skipping last 42936


...

Skipping last 61976


One chunk from each file:  86%|████████▌ | 6/7 [00:01<00:00,  4.01it/s]

Skipping last 25784
Skipping last 25176


One chunk from each file: 100%|██████████| 7/7 [00:01<00:00,  4.01it/s]

Skipping last 12760


In [4]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, 2, shuffle=False, drop_last=False)
next(iter(loader))['data'].shape

torch.Size([2, 4, 73728])

In [5]:
# device = torch.device('cuda')
# encoder = encoder.to(device)
# context_network = context_network.to(device)

batch = next(iter(loader))
batch['data'] = batch['data'] #].to(device)
batch = encoder(batch)
batch = context_network(batch, run_full=True)
batch

{'data': tensor([[[ 3.5227e-01,  3.0602e-01,  1.8820e-01,  ..., -1.8532e-01,
           -2.8521e-02,  1.1881e-01],
          [ 3.1634e-02,  3.5276e-02,  3.5589e-02,  ..., -1.7402e-01,
           -2.6424e-01, -2.0152e-01],
          [-1.2210e-01, -1.5446e-01, -1.4964e-01,  ..., -3.4669e-01,
           -2.8352e-01, -9.2478e-03],
          [ 2.0820e-01,  1.6746e-01,  6.9722e-02,  ..., -1.3563e-01,
           -2.1028e-01, -1.7068e-01]],
 
         [[ 1.7509e-01,  1.7674e-01,  2.4764e-01,  ...,  7.0802e-01,
            5.3452e-01,  3.0425e-01],
          [-9.0456e-02, -4.9613e-04,  3.0961e-02,  ...,  2.4119e-01,
            1.5493e-01,  5.2908e-02],
          [ 2.9720e-01,  5.0397e-01,  6.1361e-01,  ...,  2.9314e-01,
            3.1932e-01,  3.4552e-01],
          [-7.6991e-02,  2.6323e-02,  1.0559e-01,  ...,  5.6445e-01,
            5.8005e-01,  5.1308e-01]]]),
 'is_liked': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 2, 2, 2]]),
 'song_ids': tensor([[ 0,  0,  0,  ...,  0,  0

In [5]:
context = []
uid = []
song_id = []
is_liked = []

with torch.no_grad():
    for batch in tqdm(loader):
        batch = encoder(batch)
        batch = context_network(batch, run_full=True)
        context.append(batch['full_context_vectors'])
        uid.append(batch['user_id'])
        song_id.append(batch['song_ids'])
        is_liked.append(batch['is_liked'])

context = torch.cat(context, 0)
uid = torch.cat(uid, 0)
song_id = torch.cat(song_id, 0)
is_liked = torch.cat(is_liked, 0)

100%|██████████| 24/24 [00:23<00:00,  1.04it/s]


In [7]:
context.shape

torch.Size([47, 768, 512])

In [8]:
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6]
# [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1])


In [9]:
is_test = [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1]




In [10]:
is_liked.shape

torch.Size([47, 768])

In [22]:
uid

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6])

In [58]:
uid_per_item = uid.view(1, -1).repeat(768, 1).T.flatten()

In [ ]:
torch.arange()

In [62]:
import warnings
warnings.filterwarnings('ignore', '.*')

In [68]:
uid_per_item.min()

tensor(0)

In [79]:
song_id.flatten().shape

torch.Size([36096])

In [ ]:
uid_per_item = uid.view(1, -1).repeat(768, 1).T.flatten()
song_per_item = song_id.flatten().shape

In [85]:
song_per_item

torch.Size([36096])

In [96]:
a = song_per_item[uid_per_item == 4]
a.unique()

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36,
        37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52])

In [90]:
53 /

13.25

In [98]:
from eval.reg import evaluate_logistic_regression
import warnings
warnings.filterwarnings('ignore', '.*')

results = []

uid_per_item = uid.view(1, -1).repeat(768, 1).T.flatten()
song_per_item = song_id.flatten()

X = context.view(47 * 768, 512)
y = is_liked.view(47 * 768)
for user in range(7):
    my_x = X[uid_per_item == user]
    my_y = y[uid_per_item == user]
    my_songs = song_per_item[uid_per_item == user]
    
    not_zero = (my_y != 0)
    my_x = my_x[not_zero]
    my_y = my_y[not_zero]
    # my_songs = my_songs[not_zero]
    
    
    # for song in my_songs.unique():
    #     my_x[my_songs == song].mean(0)
    
    results.append(evaluate_logistic_regression(my_x, my_y, 'cv', do_norm=False))
    
for i in range(len(results)):
    print(i, results[i])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

0 0.9194625419864708
1 0.9828882337639817
2 0.9528329886076365
3 0.908150293150293
4 0.9342981186685962
5 0.9235962436908887
6 0.9254078083138882


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

In [99]:
for i in range(len(results)):
    print(i, results[i])

0 0.9194625419864708
1 0.9828882337639817
2 0.9528329886076365
3 0.908150293150293
4 0.9342981186685962
5 0.9235962436908887
6 0.9254078083138882


In [134]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score


def evaluate_logistic_regression_2(X, y, mode, do_norm):
    if do_norm:
        X = X / torch.norm(X, dim=1, keepdim=True)
    X, y = X.numpy(), y.numpy()
    prm = np.random.permutation(y.shape[0])
    X, y = X[prm, :], y[prm]
    
    model = LogisticRegression()
    if mode == "self":
        model.fit(X, y)
        y_pred = model.predict(X)
        return accuracy_score(y, y_pred)
    elif mode == 'cv':
        accuracies = cross_val_score(model, X, y, cv=5, scoring='accuracy', n_jobs=-1)
        return np.mean(accuracies)
    else:
        assert False

In [137]:
from eval.reg import evaluate_logistic_regression
import warnings
warnings.filterwarnings('ignore', '.*')

results = []

uid_per_item = uid.view(1, -1).repeat(768, 1).T.flatten()
song_per_item = song_id.flatten()

X = context.view(47 * 768, 512)
y = is_liked.view(47 * 768)
for user in range(7):
    my_x = X[uid_per_item == user]
    my_y = y[uid_per_item == user]
    my_songs = song_per_item[uid_per_item == user]
    
    not_zero = (my_y != 0)
    my_x = my_x[not_zero]
    my_y = my_y[not_zero]
    my_songs = my_songs[not_zero]
    
    agg_x = []
    agg_y = []
    for song in my_songs.unique():
        agg_x.append(torch.cat([
           my_x[my_songs == song].mean(0),
           my_x[my_songs == song].min(0).values,
           my_x[my_songs == song].max(0).values 
        ]))
        new_y = my_y[my_songs == song]
        assert new_y.min() == new_y.max()
        agg_y.append(new_y[0])
        
    agg_x = torch.stack(agg_x)
    agg_y = torch.tensor(agg_y)
    
    # results.append(
    print(user, evaluate_logistic_regression_2(agg_x, agg_y, 'self', do_norm=False))
    
    # OVERFIT!
    

0 1.0
1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0


In [ ]:
from eval.reg import evaluate_logistic_regression
import warnings
warnings.filterwarnings('ignore', '.*')

results = []

uid_per_item = uid.view(1, -1).repeat(768, 1).T.flatten()
song_per_item = song_id.flatten()

X = context.view(47 * 768, 512)
y = is_liked.view(47 * 768)

# TRAIN/TEST по невиденным до этого песням!

for user in range(7):
    my_x = X[uid_per_item == user]
    my_y = y[uid_per_item == user]
    my_songs = song_per_item[uid_per_item == user]
    
    not_zero = (my_y != 0)
    my_x = my_x[not_zero]
    my_y = my_y[not_zero]
    my_songs = my_songs[not_zero]
    
    agg_x = []
    agg_y = []
    for song in my_songs.unique():
        agg_x.append(torch.cat([
           my_x[my_songs == song].mean(0),
           my_x[my_songs == song].min(0).values,
           my_x[my_songs == song].max(0).values 
        ]))
        new_y = my_y[my_songs == song]
        assert new_y.min() == new_y.max()
        agg_y.append(new_y[0])
        
    agg_x = torch.stack(agg_x)
    agg_y = torch.tensor(agg_y)
    
    # results.append(
    print(user, evaluate_logistic_regression_2(agg_x, agg_y, 'cv', do_norm=False))
    
    # OVERFIT!
    

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

0.7616425988822091

In [13]:
torch.zeros(47, dtype=int)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Cleaner eval

for each person, last halve of songs - eval

In [7]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [13]:
song_id * 0

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [56]:
from eval.reg import evaluate_logistic_regression
import warnings
warnings.filterwarnings('ignore', '.*')
import numpy as np
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import accuracy_score

results = []

uid_per_item = (uid.view(-1, 1) + song_id * 0).flatten() # uid.view(1, -1).repeat(768, 1).T.flatten()
song_per_item = song_id.flatten()

X = context.view(47 * 768, 512)
y = is_liked.view(47 * 768)

r = []

for user in range(7):
    my_x = X[uid_per_item == user]
    my_y = y[uid_per_item == user]
    my_songs = song_per_item[uid_per_item == user]  # uid_per_item == user]
    
    not_zero = (my_y != 0)
    my_x = my_x[not_zero]
    my_y = my_y[not_zero]
    my_songs = my_songs[not_zero]
    
    edge = ind_from_edge(my_songs)
    # print(edge)
    my_x = my_x[edge <= 10]
    my_y = my_y[edge <= 10]
    my_songs = my_songs[edge <= 10]
    
    songs_seq = torch.unique_consecutive(my_songs).numpy()
    # np.random.shuffle(songs_seq)
    
    train_songs = songs_seq[:35]
    train_mask = np.isin(my_songs.numpy(), train_songs)
    
    # np.random.shuffle(train_mask) -- this causes score boost from 0.5/0.7 to 0.9

    
    train_x = my_x[train_mask]
    train_y = my_y[train_mask]
    test_x = my_x[~train_mask]
    test_y = my_y[~train_mask]
    
    print(train_x.shape, test_x.shape)
    
    # Train logistic regression model
    model = LogisticRegression(penalty='l1', solver='liblinear', C=0.0001) # LogisticRegression(max_iter=2000)
    model.fit(train_x, train_y)
    
    # Predict and calculate accuracy on train and test sets
    train_predictions = model.predict(train_x)
    train_accuracy = accuracy_score(train_y, train_predictions)
    print(f"User {user}: Train Accuracy = {train_accuracy:.4f}")
    
    test_predictions = model.predict(test_x)
    test_accuracy = accuracy_score(test_y, test_predictions)
    r.append(test_y == test_predictions)
    print(f"User {user}: Test Accuracy = {test_accuracy:.4f}")
print("FINAL:", (torch.cat(r) * 1.0).mean())

torch.Size([350, 512]) torch.Size([170, 512])
User 0: Train Accuracy = 0.7143
User 0: Test Accuracy = 0.4706
torch.Size([350, 512]) torch.Size([90, 512])
User 1: Train Accuracy = 0.8571
User 1: Test Accuracy = 0.6667
torch.Size([350, 512]) torch.Size([140, 512])
User 2: Train Accuracy = 0.6857
User 2: Test Accuracy = 0.9286
torch.Size([350, 512]) torch.Size([110, 512])
User 3: Train Accuracy = 0.6857
User 3: Test Accuracy = 0.5455
torch.Size([350, 512]) torch.Size([150, 512])
User 4: Train Accuracy = 0.4857
User 4: Test Accuracy = 0.7333
torch.Size([350, 512]) torch.Size([150, 512])
User 5: Train Accuracy = 0.7429
User 5: Test Accuracy = 0.6667
torch.Size([350, 512]) torch.Size([167, 512])
User 6: Train Accuracy = 0.6857
User 6: Test Accuracy = 0.3593
FINAL: tensor(0.6141)


In [53]:
(torch.cat(r) * 1.0).mean()

tensor(0.6018)

In [34]:
def ind_from_edge(data):
    # numbers = [0] * len(data)

    # # Step 2: Group indices of identical elements
    # unique_values = set(data)
    # for value in unique_values:
    #     # Find indices of current value in the data array
    #     indices = [i for i, v in enumerate(data) if v == value]
    #     # Assign numbers starting from 1 at the rightmost element
    #     for rank, idx in enumerate(reversed(indices), start=1):
    #         numbers[idx] = rank
    numbers = torch.zeros_like(data)

    # Find unique values and their inverse indices
    unique_values, inverse_indices = torch.unique(data, return_inverse=True)

    # Iterate over unique values to assign numbers
    for value in unique_values:
        # Get indices where data matches the current value
        indices = (data == value).nonzero(as_tuple=True)[0]
        # Assign ranks in reverse order
        numbers[indices] = torch.arange(len(indices), 0, -1)
            
    return numbers

In [36]:
ind_from_edge(my_songs)

tensor([62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45,
        44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27,
        26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,
         8,  7,  6,  5,  4,  3,  2,  1, 63, 62, 61, 60, 59, 58, 57, 56])

In [37]:
my_songs[:70]

tensor([34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
        34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
        34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,
        34, 34, 34, 34, 34, 34, 34, 34, 45, 45, 45, 45, 45, 45, 45, 45])